In [2]:
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [3]:
train_dir = r'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'
test_dir = r'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/test.csv'

In [4]:
train_dir

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [5]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [7]:
train_df_shuffled.shape

(7613, 5)

In [8]:
# How many examples of each class?
train_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

Target:1 (real disaster)
Text: 
Japan Marks 70th Anniversary of Hiroshima Atomic Bombing http://t.co/93vqkdFgnr
-----

Target:0 (not a real disaster)
Text: 
that new lil b x chance is nothing but flames
-----

Target:0 (not a real disaster)
Text: 
Can you recommend anyone for this #job? RN Emergency Services Full Time 3p - 3\:30a Rose de Lima Campus - http://t.co/xQrLEWiA4x #Hiring
-----

Target:1 (real disaster)
Text: 
Pak Army Helicopter crashed in Mansehra.
-----

Target:0 (not a real disaster)
Text: 
I liked a @YouTube video http://t.co/M9YdA5k6jf Spyro 3 Texture Hacks - 'Desolate Gardens' [In-game]
-----


### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [12]:
train_df['text'].shape

(7613,)

In [13]:
train_df["text"]

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [14]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [16]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [17]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

101905

In [18]:
train_sentences.shape

(6851,)

In [19]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

15

In [20]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [22]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [23]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

Original text: 

 A real life tragedy:
Monica's hair on any given season of Friends 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   3,  386,  122,  454,    1, 1192,   11,  234, 2105, 1167,    6,
         805,    0,    0,    0]], dtype=int64)>

In [24]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [25]:
len(words_in_vocab)

10000

In [26]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [27]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

In [29]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

Original text: May Allah help all those suffering from the #Pakistan floods! You and your families are in our #Dua

Vectorized text: [[ 123 1717  155   37  168 3364   20    2  890  212   12    7   35  137
    21]]

Vectorized text shape: (1, 15)
Embbedded text: [[[-0.0046602   0.02578893 -0.0443277  -0.01363412  0.02812405
    0.04619789 -0.00936579 -0.02690539  0.02482818  0.03366214
    0.02047982 -0.03055223  0.01857655 -0.00883909 -0.04435896
   -0.02487029  0.01363086 -0.01285052 -0.02063692 -0.00852848
    0.03168347  0.02547171 -0.02377809 -0.02359221  0.0403748
   -0.00997776  0.03908983 -0.0353597  -0.04443836 -0.04150976
    0.02382693 -0.01586958 -0.02827058  0.01349989 -0.0053147
    0.01633703  0.01043546 -0.049647   -0.03513365 -0.02273953
    0.03953303 -0.03813829  0.00033499 -0.02495809 -0.02868257
   -0.02139223 -0.0279941   0.0096593  -0.04466773  0.01287934
   -0.00331734 -0.03968232 -0.04359592 -0.00727409  0.01395028
    0.02114272  0.04953044 -0.0082715  -0.04917

### Model_2: Long Short Term Memory (LSTM) Model

LSTM = Long short term memory (One of the most popular LSTM cells)

Our structure of a RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output (label probability)
```

In [30]:
# Create a LTSM Model

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences = True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name ="model_2_LSTM")

In [31]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                      

In [32]:
# Compile the model
model_2.compile(loss = "binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [33]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.5237 - accuracy: 0.7393 - val_loss: 0.4663 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3180 - accuracy: 0.8701 - val_loss: 0.4768 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2203 - accuracy: 0.9168 - val_loss: 0.5420 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1594 - accuracy: 0.9428 - val_loss: 0.7136 - val_accuracy: 0.7349
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1224 - accuracy: 0.9534 - val_loss: 0.7498 - val_accuracy: 0.7402


In [34]:
# Make predictions using the model_2: LSTM Model
model_2_preds_probs = model_2.predict(val_sentences)
model_2_preds_probs

24/24 [==============================] - 1s 3ms/step


array([[4.83665476e-03],
       [1.90069959e-01],
       [4.10011038e-03],
       [5.59742302e-02],
       [6.43782616e-01],
       [1.71995223e-01],
       [8.75149912e-04],
       [1.01906983e-02],
       [5.51665090e-02],
       [9.76418734e-01],
       [7.61783481e-01],
       [3.59366685e-01],
       [5.07327914e-03],
       [1.79134905e-01],
       [1.89059064e-01],
       [2.98968349e-02],
       [9.53797758e-01],
       [3.68909887e-03],
       [9.99974012e-01],
       [1.51515799e-03],
       [3.64286184e-01],
       [7.46327817e-01],
       [1.81551859e-01],
       [3.20731938e-01],
       [1.01199009e-01],
       [2.98648626e-02],
       [9.91921961e-01],
       [8.29198420e-01],
       [9.99872208e-01],
       [6.18879159e-04],
       [2.38330327e-02],
       [1.65589433e-03],
       [6.28897667e-01],
       [4.01949972e-01],
       [1.04575492e-01],
       [7.05182850e-01],
       [1.63345113e-01],
       [5.95166795e-02],
       [1.91015960e-03],
       [1.67153537e-01],


In [35]:
model_2_preds_probs.shape

(762, 1)

In [36]:
# Convert model_2_preds_probs to the val_labels'format(0,1)
model_2_preds = tf.squeeze(tf.round(model_2_preds_probs))
model_2_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1

In [37]:
# Evaluate the model by comparing the model_2_preds to val_labels
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_2_accuracy_score = accuracy_score(val_labels, model_2_preds)
model_2_precision, model_2_recall, model_2_fscore, _ =precision_recall_fscore_support(val_labels,
                                                                                      model_2_preds,
                                                                                      average = "weighted")

In [38]:
model_2_precision

0.7397964827407357

In [39]:
dict_result = {"model_2_accuracy_score": model_2_accuracy_score,
               "model_2_precision": model_2_precision,
               "model_2_recall": {model_2_recall},
               "model_2_f1score": {model_2_fscore}}
dict_result

{'model_2_accuracy_score': 0.7401574803149606,
 'model_2_precision': 0.7397964827407357,
 'model_2_recall': {0.7401574803149606},
 'model_2_f1score': {0.7374347787537703}}